In [2]:
import pandas as pd
import numpy as np
import pymc as pm

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
flipped = pd.read_csv("https://github.com/cnickol26/BigDataBowl2023/blob/main/data_flipped.csv?raw=true")
flipped

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_75516/3063244417.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  flipped = pd.read_csv("https://github.com/cnickol26/BigDataBowl2023/blob/main/data_flipped.csv?raw=true")


,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End,Flip
0,202109090097,41.56,23.92,2021090900,97,42403.0,1,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.47,8.77,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
1,202109090097,41.56,23.92,2021090900,97,42403.0,2,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.46,8.76,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
2,202109090097,41.56,23.92,2021090900,97,42403.0,3,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.43,8.75,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
3,202109090097,41.56,23.92,2021090900,97,42403.0,4,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.42,8.74,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
4,202109090097,41.56,23.92,2021090900,97,42403.0,5,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,2021-09-10 00:26:31,2.40,8.73,none,2021-09-10 00:26:35,53441.0,42403.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629274,20211025003926,85.95,23.68,2021102500,3926,NaN,45,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.18,3.67,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629275,20211025003926,85.95,23.68,2021102500,3926,NaN,46,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.22,3.89,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629276,20211025003926,85.95,23.68,2021102500,3926,NaN,47,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.27,4.08,none,2021-10-26 03:14:08,37097.0,46083.0,0.0
1629277,20211025003926,85.95,23.68,2021102500,3926,NaN,48,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,2021-10-26 03:14:05,7.31,4.26,none,2021-10-26 03:14:08,37097.0,46083.0,0.0


In [4]:
flipped.columns

Index(['uniqueplayId', 'football_x', 'football_y', 'gameId', 'playId', 'nflId',
       'frameId', 'time', 'jerseyNumber', 'team', 'playDirection', 'x', 'y',
       's', 'a', 'dis', 'o', 'dir', 'event', 'pff_role', 'pff_positionLinedUp',
       'pff_nflIdBlockedPlayer', 'pff_blockType', 'dropBackType',
       'pff_playAction', 'ball_snap_time', 'new_x', 'new_y', 'chip_side',
       'end_time', 'Right_End', 'Left_End', 'Flip'],
      dtype='object')

In [5]:
frames_per_play = flipped[flipped['pff_positionLinedUp']=='QB'].groupby('uniqueplayId', as_index=False).size().rename(columns={'size':'play_length_frames'})

In [6]:
frames_per_play

,uniqueplayId,play_length_frames
0,202109090097,43
1,202109120056,37
2,202109120163,32
3,202109120288,27
4,202109120578,27
...,...,...
5693,20211025003536,63
5694,20211025003660,33
5695,20211025003684,53
5696,20211025003904,37


In [7]:
flipped = flipped.merge(frames_per_play, on='uniqueplayId', how='left')

In [8]:
flipped[(flipped['uniqueplayId']==2021101004676)&(flipped['jerseyNumber']==72)].T

,1410190,1410191,1410192,1410193,1410194,1410195,1410196,1410197,1410198,1410199,...,1410221,1410222,1410223,1410224,1410225,1410226,1410227,1410228,1410229,1410230
uniqueplayId,2021101004676,2021101004676,2021101004676,2021101004676,2021101004676,2021101004676,2021101004676,2021101004676,2021101004676,2021101004676,...,2021101004676,2021101004676,2021101004676,2021101004676,2021101004676,2021101004676,2021101004676,2021101004676,2021101004676,2021101004676
football_x,86.44,86.44,86.44,86.44,86.44,86.44,86.44,86.44,86.44,86.44,...,86.44,86.44,86.44,86.44,86.44,86.44,86.44,86.44,86.44,86.44
football_y,29.46,29.46,29.46,29.46,29.46,29.46,29.46,29.46,29.46,29.46,...,29.46,29.46,29.46,29.46,29.46,29.46,29.46,29.46,29.46,29.46
gameId,2021101004,2021101004,2021101004,2021101004,2021101004,2021101004,2021101004,2021101004,2021101004,2021101004,...,2021101004,2021101004,2021101004,2021101004,2021101004,2021101004,2021101004,2021101004,2021101004,2021101004
playId,676,676,676,676,676,676,676,676,676,676,...,676,676,676,676,676,676,676,676,676,676
nflId,40124.0,40124.0,40124.0,40124.0,40124.0,40124.0,40124.0,40124.0,40124.0,40124.0,...,40124.0,40124.0,40124.0,40124.0,40124.0,40124.0,40124.0,40124.0,40124.0,40124.0
frameId,5,6,7,8,9,10,11,12,13,14,...,36,37,38,39,40,41,42,43,44,45
time,2021-10-10 17:32:35,2021-10-10 17:32:35,2021-10-10 17:32:35,2021-10-10 17:32:35,2021-10-10 17:32:35,2021-10-10 17:32:35,2021-10-10 17:32:35,2021-10-10 17:32:35,2021-10-10 17:32:35,2021-10-10 17:32:35,...,2021-10-10 17:32:38,2021-10-10 17:32:38,2021-10-10 17:32:38,2021-10-10 17:32:38,2021-10-10 17:32:38,2021-10-10 17:32:38,2021-10-10 17:32:38,2021-10-10 17:32:38,2021-10-10 17:32:38,2021-10-10 17:32:39
jerseyNumber,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,...,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0
team,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,...,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN


In [9]:
flipped[(flipped['uniqueplayId']==20211003112876)&(flipped['jerseyNumber']==40)].T

,918760,918761,918762,918763,918764,918765,918766,918767,918768,918769,...,918779,918780,918781,918782,918783,918784,918785,918786,918787,918788
uniqueplayId,20211003112876,20211003112876,20211003112876,20211003112876,20211003112876,20211003112876,20211003112876,20211003112876,20211003112876,20211003112876,...,20211003112876,20211003112876,20211003112876,20211003112876,20211003112876,20211003112876,20211003112876,20211003112876,20211003112876,20211003112876
football_x,59.05,59.05,59.05,59.05,59.05,59.05,59.05,59.05,59.05,59.05,...,59.05,59.05,59.05,59.05,59.05,59.05,59.05,59.05,59.05,59.05
football_y,23.92,23.92,23.92,23.92,23.92,23.92,23.92,23.92,23.92,23.92,...,23.92,23.92,23.92,23.92,23.92,23.92,23.92,23.92,23.92,23.92
gameId,2021100311,2021100311,2021100311,2021100311,2021100311,2021100311,2021100311,2021100311,2021100311,2021100311,...,2021100311,2021100311,2021100311,2021100311,2021100311,2021100311,2021100311,2021100311,2021100311,2021100311
playId,2876,2876,2876,2876,2876,2876,2876,2876,2876,2876,...,2876,2876,2876,2876,2876,2876,2876,2876,2876,2876
nflId,52586.0,52586.0,52586.0,52586.0,52586.0,52586.0,52586.0,52586.0,52586.0,52586.0,...,52586.0,52586.0,52586.0,52586.0,52586.0,52586.0,52586.0,52586.0,52586.0,52586.0
frameId,2,3,4,5,6,7,8,9,10,11,...,21,22,23,24,25,26,27,28,29,30
time,2021-10-03 22:38:13,2021-10-03 22:38:13,2021-10-03 22:38:13,2021-10-03 22:38:13,2021-10-03 22:38:13,2021-10-03 22:38:13,2021-10-03 22:38:13,2021-10-03 22:38:13,2021-10-03 22:38:13,2021-10-03 22:38:13,...,2021-10-03 22:38:14,2021-10-03 22:38:15,2021-10-03 22:38:15,2021-10-03 22:38:15,2021-10-03 22:38:15,2021-10-03 22:38:15,2021-10-03 22:38:15,2021-10-03 22:38:15,2021-10-03 22:38:15,2021-10-03 22:38:15
jerseyNumber,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,...,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0
team,DEN,DEN,DEN,DEN,DEN,DEN,DEN,DEN,DEN,DEN,...,DEN,DEN,DEN,DEN,DEN,DEN,DEN,DEN,DEN,DEN


In [10]:
flipped['play_length_frames'].min()

19

In [11]:
flipped[flipped['play_length_frames']==19]

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,pff_playAction,ball_snap_time,new_x,new_y,chip_side,end_time,Right_End,Left_End,Flip,play_length_frames
22580,20210912103127,42.03,29.60,2021091210,3127,43854.0,6,2021-09-12 22:48:05,57.0,MIA,...,0,2021-09-12 22:48:05,0.72,4.47,none,2021-09-12 22:48:06,43321.0,43854.0,0.0,19
22581,20210912103127,42.03,29.60,2021091210,3127,43854.0,7,2021-09-12 22:48:05,57.0,MIA,...,0,2021-09-12 22:48:05,0.66,4.45,none,2021-09-12 22:48:06,43321.0,43854.0,0.0,19
22582,20210912103127,42.03,29.60,2021091210,3127,43854.0,8,2021-09-12 22:48:05,57.0,MIA,...,0,2021-09-12 22:48:05,0.54,4.39,none,2021-09-12 22:48:06,43321.0,43854.0,0.0,19
22583,20210912103127,42.03,29.60,2021091210,3127,43854.0,9,2021-09-12 22:48:05,57.0,MIA,...,0,2021-09-12 22:48:05,0.39,4.32,none,2021-09-12 22:48:06,43321.0,43854.0,0.0,19
22584,20210912103127,42.03,29.60,2021091210,3127,43854.0,10,2021-09-12 22:48:05,57.0,MIA,...,0,2021-09-12 22:48:05,0.19,4.22,none,2021-09-12 22:48:06,43321.0,43854.0,0.0,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564582,20211024002641,41.30,23.72,2021102400,2641,NaN,20,2021-10-24 19:15:47,NaN,football,...,0,2021-10-24 19:15:46,5.52,-1.09,none,2021-10-24 19:15:47,46146.0,44915.0,0.0,19
1564583,20211024002641,41.30,23.72,2021102400,2641,NaN,21,2021-10-24 19:15:47,NaN,football,...,0,2021-10-24 19:15:46,4.83,-4.73,none,2021-10-24 19:15:47,46146.0,44915.0,0.0,19
1564584,20211024002641,41.30,23.72,2021102400,2641,NaN,22,2021-10-24 19:15:47,NaN,football,...,0,2021-10-24 19:15:46,4.64,-6.57,none,2021-10-24 19:15:47,46146.0,44915.0,0.0,19
1564585,20211024002641,41.30,23.72,2021102400,2641,NaN,23,2021-10-24 19:15:47,NaN,football,...,0,2021-10-24 19:15:46,4.46,-8.40,none,2021-10-24 19:15:47,46146.0,44915.0,0.0,19


In [12]:
flipped.columns

Index(['uniqueplayId', 'football_x', 'football_y', 'gameId', 'playId', 'nflId',
       'frameId', 'time', 'jerseyNumber', 'team', 'playDirection', 'x', 'y',
       's', 'a', 'dis', 'o', 'dir', 'event', 'pff_role', 'pff_positionLinedUp',
       'pff_nflIdBlockedPlayer', 'pff_blockType', 'dropBackType',
       'pff_playAction', 'ball_snap_time', 'new_x', 'new_y', 'chip_side',
       'end_time', 'Right_End', 'Left_End', 'Flip', 'play_length_frames'],
      dtype='object')

In [93]:
def name_func(pos):
    temp = {'nflId':pos+'_nflId',
     'jerseyNumber':pos+'_jerseyNumber',
     'team':pos+'_team', 
     'x':pos+'_x', 
     'y':pos+'_y',
     's':pos+'_s', 
     'a':pos+'_a', 
     'dis':pos+'_dis', 
     'o':pos+'_o', 
     'dir':pos+'_dir', 
     'pff_role':pos+'_pff_role', 
     'pff_positionLinedUp':pos+'_pff_positionLinedUp',
     'pff_nflIdBlockedPlayer':pos+'_pff_nflIdBlockedPlayer', 
     'pff_blockType':pos+'_pff_blockType', 
     'new_x':pos+'_new_x', 
     'new_y':pos+'_new_y', 
     'Flip':pos+'_Flip'}
    return temp

In [94]:
qb = flipped[(flipped['pff_positionLinedUp']=='QB')]
qb = qb.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('QB')).reset_index(drop=True)

In [95]:
qb_flip = flipped[(flipped['pff_positionLinedUp']=='QB_flip')]
qb_flip = qb_flip.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('QB_flip')).reset_index(drop=True)

In [96]:
ball = flipped[(flipped['pff_positionLinedUp']=='ball')]
ball = ball.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('ball')).reset_index(drop=True)

In [97]:
ball_flip = flipped[(flipped['pff_positionLinedUp']=='ball_flip')]
ball_flip = ball_flip.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('ball_flip')).reset_index(drop=True)


In [98]:
rt = flipped[(flipped['pff_positionLinedUp']=='RT')]
rt = rt.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('RT')).reset_index(drop=True)

In [99]:
lt = flipped[(flipped['pff_positionLinedUp']=='LT')]
lt = lt.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('LT')).reset_index(drop=True)

In [100]:
le = flipped[((flipped['pff_role']=='Pass Rush')|(flipped['pff_role']=='Coverage'))&
             (flipped['Flip']==1)]
le = le.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('LE')).reset_index(drop=True)

In [101]:
re = flipped[((flipped['pff_role']=='Pass Rush')|(flipped['pff_role']=='Coverage'))&
             (flipped['Flip']==0)]
re = re.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('RE')).reset_index(drop=True)

In [102]:
play = flipped[(flipped['pff_positionLinedUp']=='QB')][['uniqueplayId', 'football_x', 'football_y', 
                                                        'gameId', 'playId', 'frameId', 'time', 'playDirection', 
                                                        'event', 'ball_snap_time', 'end_time', 'Right_End', 
                                                        'Left_End', 'play_length_frames']].reset_index(drop=True)

In [103]:
final = play.merge(qb, on=['uniqueplayId', 'frameId'], how='left').merge(qb_flip, on=['uniqueplayId', 'frameId'], how='left')

In [104]:
final = final.merge(ball, on=['uniqueplayId', 'frameId'], how='left').merge(ball_flip, on=['uniqueplayId', 'frameId'], how='left')

In [105]:
final = final.merge(rt, on=['uniqueplayId', 'frameId'], how='left').merge(lt, on=['uniqueplayId', 'frameId'], how='left')

In [106]:
final = final.merge(re, on=['uniqueplayId', 'frameId'], how='left').merge(le, on=['uniqueplayId', 'frameId'], how='left')

In [107]:
final

,uniqueplayId,football_x,football_y,gameId,playId,frameId,time,playDirection,event,ball_snap_time,...,LE_dis,LE_o,LE_dir,LE_pff_role,LE_pff_positionLinedUp,LE_pff_nflIdBlockedPlayer,LE_pff_blockType,LE_new_x,LE_new_y,LE_Flip
0,202109090097,41.56,23.92,2021090900,97,1,2021-09-10 00:26:31,right,None,2021-09-10 00:26:31,...,0.01,276.51,290.12,Pass Rush,LILB,NaN,NaN,2.15,2.04,1.0
1,202109090097,41.56,23.92,2021090900,97,2,2021-09-10 00:26:31,right,None,2021-09-10 00:26:31,...,0.00,276.51,289.82,Pass Rush,LILB,NaN,NaN,2.15,2.04,1.0
2,202109090097,41.56,23.92,2021090900,97,3,2021-09-10 00:26:31,right,None,2021-09-10 00:26:31,...,0.01,277.16,296.04,Pass Rush,LILB,NaN,NaN,2.15,2.03,1.0
3,202109090097,41.56,23.92,2021090900,97,4,2021-09-10 00:26:31,right,None,2021-09-10 00:26:31,...,0.02,277.16,308.81,Pass Rush,LILB,NaN,NaN,2.14,2.01,1.0
4,202109090097,41.56,23.92,2021090900,97,5,2021-09-10 00:26:31,right,None,2021-09-10 00:26:31,...,0.01,277.16,325.47,Pass Rush,LILB,NaN,NaN,2.13,2.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214235,20211025003926,85.95,23.68,2021102500,3926,45,2021-10-26 03:14:08,left,None,2021-10-26 03:14:05,...,0.19,113.22,26.59,Pass Rush,LOLB,NaN,NaN,4.83,3.77,1.0
214236,20211025003926,85.95,23.68,2021102500,3926,46,2021-10-26 03:14:08,left,None,2021-10-26 03:14:05,...,0.19,113.22,23.83,Pass Rush,LOLB,NaN,NaN,4.91,3.94,1.0
214237,20211025003926,85.95,23.68,2021102500,3926,47,2021-10-26 03:14:09,left,None,2021-10-26 03:14:05,...,0.18,113.88,21.57,Pass Rush,LOLB,NaN,NaN,4.98,4.11,1.0
214238,20211025003926,85.95,23.68,2021102500,3926,48,2021-10-26 03:14:09,left,None,2021-10-26 03:14:05,...,0.18,115.29,18.94,Pass Rush,LOLB,NaN,NaN,5.05,4.28,1.0


In [111]:
flipped[((flipped['pff_role']=='Pass Rush')|(flipped['pff_role']=='Coverage'))&
             (flipped['Flip']==1)]
le = le.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_time',
       'end_time', 'Right_End', 'Left_End', 'play_length_frames', 
       'dropBackType', 'pff_playAction', 'chip_side'], axis=1).rename(columns=name_func('LE')).reset_index(drop=True)

array([nan])

In [118]:
flipped[((flipped['pff_role']=='Pass Rush')|(flipped['pff_role']=='Coverage'))&
             (flipped['Flip']==1)].T

,807558,807559,807560,807561,807562,807563,807564,807565,807566,807567,...,1002999,1003000,1003001,1003002,1003003,1003004,1003005,1003006,1003007,1003008
uniqueplayId,202109090097,202109090097,202109090097,202109090097,202109090097,202109090097,202109090097,202109090097,202109090097,202109090097,...,20211025003926,20211025003926,20211025003926,20211025003926,20211025003926,20211025003926,20211025003926,20211025003926,20211025003926,20211025003926
football_x,41.56,41.56,41.56,41.56,41.56,41.56,41.56,41.56,41.56,41.56,...,85.95,85.95,85.95,85.95,85.95,85.95,85.95,85.95,85.95,85.95
football_y,23.92,23.92,23.92,23.92,23.92,23.92,23.92,23.92,23.92,23.92,...,23.68,23.68,23.68,23.68,23.68,23.68,23.68,23.68,23.68,23.68
gameId,2021090900,2021090900,2021090900,2021090900,2021090900,2021090900,2021090900,2021090900,2021090900,2021090900,...,2021102500,2021102500,2021102500,2021102500,2021102500,2021102500,2021102500,2021102500,2021102500,2021102500
playId,97,97,97,97,97,97,97,97,97,97,...,3926,3926,3926,3926,3926,3926,3926,3926,3926,3926
nflId,53441.0,53441.0,53441.0,53441.0,53441.0,53441.0,53441.0,53441.0,53441.0,53441.0,...,37097.0,37097.0,37097.0,37097.0,37097.0,37097.0,37097.0,37097.0,37097.0,37097.0
frameId,1,2,3,4,5,6,7,8,9,10,...,40,41,42,43,44,45,46,47,48,49
time,2021-09-10 00:26:31,2021-09-10 00:26:31,2021-09-10 00:26:31,2021-09-10 00:26:31,2021-09-10 00:26:31,2021-09-10 00:26:31,2021-09-10 00:26:31,2021-09-10 00:26:31,2021-09-10 00:26:31,2021-09-10 00:26:32,...,2021-10-26 03:14:08,2021-10-26 03:14:08,2021-10-26 03:14:08,2021-10-26 03:14:08,2021-10-26 03:14:08,2021-10-26 03:14:08,2021-10-26 03:14:08,2021-10-26 03:14:09,2021-10-26 03:14:09,2021-10-26 03:14:09
jerseyNumber,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,...,94.0,94.0,94.0,94.0,94.0,94.0,94.0,94.0,94.0,94.0
team,DAL,DAL,DAL,DAL,DAL,DAL,DAL,DAL,DAL,DAL,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO


In [115]:
flipped.columns

Index(['uniqueplayId', 'football_x', 'football_y', 'gameId', 'playId', 'nflId',
       'frameId', 'time', 'jerseyNumber', 'team', 'playDirection', 'x', 'y',
       's', 'a', 'dis', 'o', 'dir', 'event', 'pff_role', 'pff_positionLinedUp',
       'pff_nflIdBlockedPlayer', 'pff_blockType', 'dropBackType',
       'pff_playAction', 'ball_snap_time', 'new_x', 'new_y', 'chip_side',
       'end_time', 'Right_End', 'Left_End', 'Flip', 'play_length_frames'],
      dtype='object')

In [119]:
flipped['pff_nflIdBlockedPlayer']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
1629274   NaN
1629275   NaN
1629276   NaN
1629277   NaN
1629278   NaN
Name: pff_nflIdBlockedPlayer, Length: 1629279, dtype: float64